<a href="https://colab.research.google.com/github/Deekshith1064/Deekshith.K/blob/main/AIML_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PART_1 Game Strategy

In [3]:
from collections import namedtuple, Counter, defaultdict
import random
import math
import functools
cache = functools.lru_cache(10**6)


class Game:
    """A game is similar to a problem, but it has a terminal test instead of
    a goal test, and a utility for each terminal state. To create a game,
    subclass this class and implement actions, result, is_terminal,
    and utility. You will also need to set the .initial attribute to the
    initial state; this can be done in the constructor."""

    def actions(self, state):
        """Return a collection of the allowable moves from this state."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def is_terminal(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError


def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game. strategies is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    state = game.initial
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](game, state)
        state = game.result(state, move)
        if verbose:
            print('Player', player, 'move:', move)
            print(state)
    return state

## Part - 02

In [4]:
def minimax_search(game, state):
    """Search game tree to determine best move; return (value, move) pair."""

    player = state.to_move

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
        return v, move

    return max_value(state)

infinity = math.inf


def alphabeta_search(game, state):
    """Search game to determine best action; use alpha-beta pruning.
    ""Search all the way to the leaves."""

    player = state.to_move

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -infinity, +infinity)

## Part - 03

In [13]:
import random
from collections import defaultdict
from typing import Dict, Tuple, List, Callable

class TicTacToe:
    """Play TicTacToe on a height by width board, needing k in a row to win."""

    def init(self, height=3, width=3, k=3):
        self.k = k  # k in a row to win
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(width=width, height=height, to_move='X', utility=0)

    def actions(self, board: 'Board') -> set:
        """Legal moves are any square not yet taken."""
        return self.squares - set(board)

    def result(self, board: 'Board', square: Tuple[int, int]) -> 'Board':
        """Place a marker for the current player on square."""
        player = board.to_move
        new_board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        if self.k_in_row(new_board, player, square):
            new_board.utility = 1 if player == 'X' else -1
        return new_board

    def utility(self, board: 'Board', player: str) -> int:
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board: 'Board') -> bool:
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)

    def display(self, board: 'Board'):
        print(board)

    def k_in_row(self, board: 'Board', player: str, square: Tuple[int, int]) -> bool:
        """True if player has k pieces in a line through square."""
        def in_row(x: int, y: int, dx: int, dy: int) -> int:
            count = 0
            while 0 <= x < board.width and 0 <= y < board.height and board[x, y] == player:
                count += 1
                x += dx
                y += dy
            return count

        x, y = square
        return any(
            in_row(x, y, dx, dy) + in_row(x, y, -dx, -dy) - 1 >= self.k
            for dx, dy in ((0, 1), (1, 0), (1, 1), (1, -1))
        )

Class Board

In [14]:
class Board(defaultdict):
    """A board has the player to move, a cached utility value, and a dict of {(x, y): player} entries."""

    empty = '.'
    off = '#'

    def init(self, width=8, height=8, to_move=None, **kwds):
        super().init(self.empty)
        self.update(width=width, height=height, to_move=to_move, **kwds)
        self.utility = 0

    def new(self, changes: Dict[Tuple[int, int], str], **kwds) -> 'Board':
        """Given a dict of {(x, y): contents} changes, return a new Board with the changes."""
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def missing(self, loc: Tuple[int, int]) -> str:
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off

    def hash(self) -> int:
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

    def repr(self) -> str:
        def row(y: int) -> str:
            return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(row(y) for y in range(self.height)) + '\n'

Player Function

In [15]:
def random_player(game: TicTacToe, state: Board) -> Tuple[int, int]:
    """Random player that returns a random legal move."""
    return random.choice(list(game.actions(state)))

def player(search_algorithm: Callable[[TicTacToe, Board], Tuple[int, int]]) -> Callable[[TicTacToe, Board], Tuple[int, int]]:
    """A game player who uses the specified search algorithm."""
    return lambda game, state: search_algorithm(game, state)[1]

## Part - 04

In [16]:
import random
import math

class TicTacToe:
    def init(self):
        self.board = [' '] * 9
        self.current_winner = None  # Keep track of the winner!

    def print_board(self):
        for i in range(0, 9, 3):
            print('|'.join(self.board[i:i + 3]))
        print()

    def available_moves(self):
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def make_move(self, square, letter):
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def winner(self, square, letter):
        row_ind = square // 3
        row = self.board[row_ind * 3: (row_ind + 1) * 3]
        if all([spot == letter for spot in row]):
            return True
        col_ind = square % 3
        column = [self.board[col_ind + i * 3] for i in range(3)]
        if all([spot == letter for spot in column]):
            return True
        if square % 2 == 0:
            diagonal1 = [self.board[i] for i in [0, 4, 8]]
            if all([spot == letter for spot in diagonal1]):
                return True
            diagonal2 = [self.board[i] for i in [2, 4, 6]]
            if all([spot == letter for spot in diagonal2]):
                return True
        return False

    def is_full(self):
        return ' ' not in self.board

    def reset(self):
        self.init()

def play_game(game, players, verbose=False):
    game.reset()
    turn = 'X'
    while not game.is_full():
        if verbose:
            game.print_board()
        move = players[turn](game)
        game.make_move(move, turn)
        if game.current_winner:
            if verbose:
                game.print_board()
            return turn
        turn = 'O' if turn == 'X' else 'X'
    if verbose:
        game.print_board()
    return 'Tie'

Implementing

In [17]:
def random_player(game):
    return random.choice(game.available_moves())

Implementing

In [18]:
def alpha_beta_search(game, depth, alpha, beta, maximizing_player):
    if game.current_winner == 'X':
        return 1
    if game.current_winner == 'O':
        return -1
    if game.is_full() or depth == 0:
        return 0

    if maximizing_player:
        max_eval = -math.inf
        for move in game.available_moves():
            game.make_move(move, 'X')
            eval = alpha_beta_search(game, depth - 1, alpha, beta, False)
            game.board[move] = ' '
            game.current_winner = None
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = math.inf
        for move in game.available_moves():
            game.make_move(move, 'O')
            eval = alpha_beta_search(game, depth - 1, alpha, beta, True)
            game.board[move] = ' '
            game.current_winner = None
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

def alpha_beta_player(game):
    best_move = None
    best_value = -math.inf
    for move in game.available_moves():
        game.make_move(move, 'X')
        move_value = alpha_beta_search(game, 3, -math.inf, math.inf, False)
        game.board[move] = ' '
        game.current_winner = None
        if move_value > best_value:
            best_value = move_value
            best_move = move
    return best_move

Minimax

In [19]:
def minimax_search(game, depth, maximizing_player):
    if game.current_winner == 'X':
        return 1
    if game.current_winner == 'O':
        return -1
    if game.is_full() or depth == 0:
        return 0

    if maximizing_player:
        best_value = -math.inf
        for move in game.available_moves():
            game.make_move(move, 'X')
            move_value = minimax_search(game, depth - 1, False)
            game.board[move] = ' '
            game.current_winner = None
            best_value = max(best_value, move_value)
        return best_value
    else:
        best_value = math.inf
        for move in game.available_moves():
            game.make_move(move, 'O')
            move_value = minimax_search(game, depth - 1, True)
            game.board[move] = ' '
            game.current_winner = None
            best_value = min(best_value, move_value)
        return best_value

def minimax_player(game):
    best_move = None
    best_value = -math.inf
    for move in game.available_moves():
        game.make_move(move, 'X')
        move_value = minimax_search(game, 3, False)
        game.board[move] = ' '
        game.current_winner = None
        if move_value > best_value:
            best_value = move_value
            best_move = move
    return best_move

Running

In [20]:
# Run Tic-Tac-Toe with Alpha-Beta search for 'O' and Random Player for 'X'
result1 = play_game(TicTacToe(), {'X': random_player, 'O': alpha_beta_player}, verbose=True)
print(f"Result with Alpha-Beta vs Random: {result1}")

# Run Tic-Tac-Toe with Minimax search for 'O' and Alpha-Beta search for 'X'
result2 = play_game(TicTacToe(), {'X': alpha_beta_player, 'O': minimax_player}, verbose=True)
print(f"Result with Minimax vs Alpha-Beta: {result2}")

 | | 
 | | 
 | | 

 | | 
X| | 
 | | 

O| | 
X| | 
 | | 

O| | 
X| | 
 |X| 

O| | 
X|O| 
 |X| 

O| | 
X|O| 
 |X|X

O| |O
X|O| 
 |X|X

O|X|O
X|O| 
 |X|X

O|X|O
X|O| 
O|X|X

Result with Alpha-Beta vs Random: O
 | | 
 | | 
 | | 

X| | 
 | | 
 | | 

X|O| 
 | | 
 | | 

X|O|X
 | | 
 | | 

X|O|X
 |O| 
 | | 

X|O|X
 |O| 
 |X| 

X|O|X
 |O| 
O|X| 

X|O|X
X|O| 
O|X| 

X|O|X
X|O|O
O|X| 

X|O|X
X|O|O
O|X|X

Result with Minimax vs Alpha-Beta: Tie
